# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [4]:
# imports

import os
import requests
from typing import Optional

try:
    from openai import OpenAI  # optional, only if you have an API key
except ImportError:  # no openai package installed
    OpenAI = None

In [5]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [6]:
# set up environment

# OpenAI client (reads OPENAI_API_KEY from your environment)
# If there is no API key (or no OpenAI client library), we will just fall back to Ollama.
if OpenAI is not None and os.environ.get("OPENAI_API_KEY"):
    client_gpt = OpenAI()
else:
    client_gpt = None

# Ollama endpoint (defaults to local instance)
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")


def ask_llama(question: str, model: str = MODEL_LLAMA) -> str:
    """Ask a local Ollama Llama model a question and return the answer.

    Requires an Ollama server running locally with the model pulled.
    """
    payload = {
        "model": model,
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful senior software engineer. Explain technical concepts clearly and concisely.",
            },
            {"role": "user", "content": question},
        ],
        "stream": False,
    }

    resp = requests.post(f"{OLLAMA_BASE_URL}/api/chat", json=payload, timeout=60)
    resp.raise_for_status()
    data = resp.json()

    # Ollama chat API typically returns a 'message' object with 'content'
    content: Optional[str] = None
    message = data.get("message")
    if isinstance(message, dict):
        content = message.get("content")

    # Fallback in case of a different schema
    if content is None and "choices" in data:
        choice = data["choices"][0]
        content = choice.get("message", {}).get("content")

    if content is None:
        content = str(data)

    print(content)
    return content


def ask_gpt(question: str, model: str = MODEL_GPT) -> str:
    """Ask an OpenAI GPT model if configured, otherwise fall back to Ollama.

    This lets the notebook run even when you only have a local Ollama setup
    and no OpenAI API key.
    """
    if client_gpt is None:
        print("No OpenAI API key found; using Ollama instead of GPT.")
        return ask_llama(question, model=MODEL_LLAMA)

    stream = client_gpt.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful senior software engineer. Explain technical concepts clearly and concisely.",
            },
            {"role": "user", "content": question},
        ],
        stream=True,
    )

    full_answer = ""
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""
        print(delta, end="", flush=True)
        full_answer += delta

    print()  # final newline
    return full_answer

In [7]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [8]:
# Get gpt-4o-mini to answer, with streaming

gpt_answer = ask_gpt(question, model=MODEL_GPT)
gpt_answer

No OpenAI API key found; using Ollama instead of GPT.
**Code Explanation**

The given code snippet uses the `yield from` keyword, which is a part of Python 3.3 and later versions.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

This code generates an iterator that yields the values from a dictionary comprehension. Here's how it works:

1. `for book in books`: This iterates over each item (a dictionary) in the `books` collection.
2. `if book.get("author")`: This filters out dictionaries that do not have an "author" key or whose value for that key is empty (`None`, `False`, etc.). The `get()` method returns `None` if the key does not exist, so this effectively skips any books without authors.
3. `{book.get("author")}`: For each dictionary that passes the filter, this generates a new dictionary containing only the value associated with the "author" key.

Now, `yield from ...`: This is where it gets interesting. When you use `yield from`, it essentia

'**Code Explanation**\n\nThe given code snippet uses the `yield from` keyword, which is a part of Python 3.3 and later versions.\n\n```python\nyield from {book.get("author") for book in books if book.get("author")}\n```\n\nThis code generates an iterator that yields the values from a dictionary comprehension. Here\'s how it works:\n\n1. `for book in books`: This iterates over each item (a dictionary) in the `books` collection.\n2. `if book.get("author")`: This filters out dictionaries that do not have an "author" key or whose value for that key is empty (`None`, `False`, etc.). The `get()` method returns `None` if the key does not exist, so this effectively skips any books without authors.\n3. `{book.get("author")}`: For each dictionary that passes the filter, this generates a new dictionary containing only the value associated with the "author" key.\n\nNow, `yield from ...`: This is where it gets interesting. When you use `yield from`, it essentially delegates to the next iterable (in

In [9]:
# Get Llama 3.2 to answer

llama_answer = ask_llama(question, model=MODEL_LLAMA)
llama_answer

**Code Explanation**

This is a Python expression that uses a combination of generator expressions, list comprehensions, and the `yield from` keyword.

Here's a breakdown:

1. `{...}` - This defines a dictionary comprehension.
2. `{book.get("author") for book in books if book.get("author")}` - This is another dictionary comprehension, but it generates values instead of keys.
	* `(book.get("author"))` - For each item `book` in the collection `books`, get the value associated with the key `"author"`.
	* `for book in books` - Iterate over each item in the `books` collection.
	* `if book.get("author")` - Only include items where the `"author"` key exists and has a non-empty value.

**The `yield from` keyword**

Now, here comes the interesting part:

`yield from {...}`

This is used to "forward" values generated by an inner iterable (in this case, another generator expression).

When we use `yield from`, it yields each value generated by the inner iterable, one at a time, and passes control

'**Code Explanation**\n\nThis is a Python expression that uses a combination of generator expressions, list comprehensions, and the `yield from` keyword.\n\nHere\'s a breakdown:\n\n1. `{...}` - This defines a dictionary comprehension.\n2. `{book.get("author") for book in books if book.get("author")}` - This is another dictionary comprehension, but it generates values instead of keys.\n\t* `(book.get("author"))` - For each item `book` in the collection `books`, get the value associated with the key `"author"`.\n\t* `for book in books` - Iterate over each item in the `books` collection.\n\t* `if book.get("author")` - Only include items where the `"author"` key exists and has a non-empty value.\n\n**The `yield from` keyword**\n\nNow, here comes the interesting part:\n\n`yield from {...}`\n\nThis is used to "forward" values generated by an inner iterable (in this case, another generator expression).\n\nWhen we use `yield from`, it yields each value generated by the inner iterable, one at a